In [136]:
import os
import json
import pandas as pd
import traceback

In [137]:
from langchain.chat_models import ChatOpenAI

In [138]:
from dotenv import load_dotenv
load_dotenv()

True

In [139]:
api_key=os.getenv('open_ai_key')

In [140]:
llm=ChatOpenAI(openai_api_key=api_key,model_name='gpt-3.5-turbo',temperature=0.5)

In [141]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [142]:
RESPONSE_JSON = { 
                 "1": { "mcq": "multiple choice question", 
                       "options": { "a": "choice here",
                                   "b": "choice here", 
                                   "C": "choice here", 
                                   "d": "choice here", }
                       , "correct": "correct answer", },
                 "2": { "mcq": "multiple choice question", 
                       "options": { "a": "choice here",
                                   "b": "choice here", 
                                   "C": "choice here", 
                                   "d": "choice here", }
                       , "correct": "correct answer", },
                 "3": { "mcq": "multiple choice question", 
                       "options": { "a": "choice here",
                                   "b": "choice here", 
                                   "C": "choice here", 
                                   "d": "choice here", }
                       , "correct": "correct answer", },
                 
                 }

In [143]:
mcq_prompt = PromptTemplate(
    input_variables=['number', 'response_json', 'subject', 'text', 'tone'],
    template="""
    Text: {text}
    You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
    Make sure the questions are not repeated and check all the questions to be conforming the text as well.
    Make sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs.
    ### RESPONSE_JSON
    {response_json}
    """
)

In [144]:
quiz_chain=LLMChain(llm=llm,prompt=mcq_prompt,output_key='quiz',verbose=True)

In [145]:
evaluation_prompt = PromptTemplate(
    input_variables=['quiz', 'subject'],
    template="""
    You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students,
    you need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
    If the quiz is not at par with the cognitive and analytical abilities of the students,
    update the quiz questions which need to be changed and change the tone such that it perfectly fits the student's abilities.
    Quiz_MCQs: {quiz}
    Check from an expert English Writer of the above quiz:
    """
)

In [146]:
review_chain=LLMChain(llm=llm,prompt=evaluation_prompt,output_key='review',verbose=True)

In [147]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=['number', 'response_json', 'subject', 'text', 'tone'],output_variables=['quiz','review'])

In [148]:
generate_evaluate_chain

SequentialChain(chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\n    Text: {text}\n    You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.\n    Make sure the questions are not repeated and check all the questions to be conforming the text as well.\n    Make sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs.\n    ### RESPONSE_JSON\n    {response_json}\n    '), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021496AF0610>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021496B73290>, temperature=0.5, openai_api_key='sk-proj-xyYlwlT8Ozf8VnApBsPBT3BlbkFJqMDbaALTSQUXfZjFTrpa', openai_proxy=''), output_key='quiz'), LLMChain(verbose=True, prompt=PromptTemplate(input_variables

In [149]:
file_path='F:\MCQ_GEN_LLM\data.txt'

In [150]:
with open(file_path,'r') as file:
    data=file.read()

In [151]:
data

'Data science and data analysis are both important disciplines in the field of data management and analysis, but they differ in several key ways. While both fields involve working with data, data science is more of an interdisciplinary field that involves the application of statistical, computational, and machine learning methods to extract insights from data and make predictions, while data analysis is more focused on the examination and interpretation of data to identify patterns and trends.[37][38]\n\nData analysis typically involves working with smaller, structured datasets to answer specific questions or solve specific problems. This can involve tasks such as data cleaning, data visualization, and exploratory data analysis to gain insights into the data and develop hypotheses about relationships between variables. Data analysts typically use statistical methods to test these hypotheses and draw conclusions from the data. For example, a data analyst might analyze sales data to iden

In [152]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "C": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "C": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "C": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [153]:
Number=5
Subject='data analysis'
Tone='Hard'

In [154]:
input_data = {
    'number': Number,
    'response_json': json.dumps(RESPONSE_JSON),
    'subject': Subject,
    'text': data,
    'tone': Tone
}

In [155]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(input_data)



> Entering new LLMChain chain...
Prompt after formatting:

    Text: Data science and data analysis are both important disciplines in the field of data management and analysis, but they differ in several key ways. While both fields involve working with data, data science is more of an interdisciplinary field that involves the application of statistical, computational, and machine learning methods to extract insights from data and make predictions, while data analysis is more focused on the examination and interpretation of data to identify patterns and trends.[37][38]

Data analysis typically involves working with smaller, structured datasets to answer specific questions or solve specific problems. This can involve tasks such as data cleaning, data visualization, and exploratory data analysis to gain insights into the data and develop hypotheses about relationships between variables. Data analysts typically use statistical methods to test these hypotheses and draw conclusions from th

In [156]:
response

{'number': 5,
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "C": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "C": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "C": "choice here", "d": "choice here"}, "correct": "correct answer"}}',
 'subject': 'data analysis',
 'text': 'Data science and data analysis are both important disciplines in the field of data management and analysis, but they differ in several key ways. While both fields involve working with data, data science is more of an interdisciplinary field that involves the application of statistical, computational, and machine learning methods to extract insights from data and make predictions, while data analysis is more focused on the examina

In [157]:
print(f"total tokens: ",cb.total_tokens)
print(f"Prompt tokens: ",cb.prompt_tokens)
print(f"completion tokens: ",cb.completion_tokens)
print(f"total cost: ",cb.total_cost)

total tokens:  1759
Prompt tokens:  1321
completion tokens:  438
total cost:  0.0028575000000000007


In [158]:
quiz=response.get('quiz')

In [159]:
ans=json.loads(quiz)

In [161]:
ans

{'1': {'mcq': 'What is the main focus of data analysis?',
  'options': {'a': 'Building predictive models',
   'b': 'Examining and interpreting data',
   'c': 'Working with unstructured data',
   'd': 'Implementing machine learning algorithms'},
  'correct': 'b'},
 '2': {'mcq': 'What type of datasets does data analysis typically work with?',
  'options': {'a': 'Small, structured datasets',
   'b': 'Large, complex datasets',
   'c': 'Unstructured data',
   'd': 'Text or images'},
  'correct': 'a'},
 '3': {'mcq': 'What is a common task in data analysis to gain insights into the data?',
  'options': {'a': 'Data preprocessing',
   'b': 'Feature engineering',
   'c': 'Data visualization',
   'd': 'Model selection'},
  'correct': 'c'},
 '4': {'mcq': 'What is the main goal of data science?',
  'options': {'a': 'Identifying patterns and trends',
   'b': 'Making informed decisions',
   'c': 'Building predictive models',
   'd': 'Data cleaning'},
  'correct': 'c'},
 '5': {'mcq': 'What is a key di

In [163]:
quiz_data=[]
for key,value in ans.items():
    mcq=value['mcq']
    options=" | ".join(
        [
            f"option : {options}"
            for option,options in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_data.append({
        "mcq":mcq,
        "Choices":options,
        "Answer":correct
    })

In [164]:
quiz_data

[{'mcq': 'What is the main focus of data analysis?',
  'Choices': 'option : Building predictive models | option : Examining and interpreting data | option : Working with unstructured data | option : Implementing machine learning algorithms',
  'Answer': 'b'},
 {'mcq': 'What type of datasets does data analysis typically work with?',
  'Choices': 'option : Small, structured datasets | option : Large, complex datasets | option : Unstructured data | option : Text or images',
  'Answer': 'a'},
 {'mcq': 'What is a common task in data analysis to gain insights into the data?',
  'Choices': 'option : Data preprocessing | option : Feature engineering | option : Data visualization | option : Model selection',
  'Answer': 'c'},
 {'mcq': 'What is the main goal of data science?',
  'Choices': 'option : Identifying patterns and trends | option : Making informed decisions | option : Building predictive models | option : Data cleaning',
  'Answer': 'c'},
 {'mcq': 'What is a key difference between data

In [165]:
df=pd.DataFrame(quiz_data)
df

,mcq,Choices,Answer
0,What is the main focus of data analysis?,option : Building predictive models | option :...,b
1,What type of datasets does data analysis typic...,"option : Small, structured datasets | option :...",a
2,What is a common task in data analysis to gain...,option : Data preprocessing | option : Feature...,c
3,What is the main goal of data science?,option : Identifying patterns and trends | opt...,c
4,What is a key difference between data analysis...,option : Data analysis uses machine learning a...,d


In [167]:
df.to_csv('quiz.csv',index=False)